In [ ]:
import scipy.io as sio

# ourdata = sio.loadmat("/Users/xupeng.tong/Documents/Data/OriginalData/B_mean_2labels.mat")
ourdata = sio.loadmat("/Volumes/TONY/Regeneron/Data/OriginalData/B_mean_2labels.mat")

inputX = ourdata['X']
# inputX = normalize(inputX, axis=0)
inputY = ourdata['Y'][0,:]
columnNames = ourdata['columnNames']

In [ ]:
a = np.random.choice(len(c_init), 100, replace=False)

In [ ]:
c_init[[2,2],[1,2]].shape

In [ ]:
from dsc import DeepSubspaceClustering
from sp import getSparcityPrior
import numpy as np

c_init = np.full((inputX.shape[1],inputX.shape[1]), 0.1, dtype=np.float32)
np.fill_diagonal(c_init, 0)

C = getSparcityPrior(inputX, C_init=c_init, optimizer='Adam', lambda1=0.01, lambda2=50, epochs=500, learning_rate=0.1,print_step=1)

In [1]:
from dsc import DeepSubspaceClustering
from sp import getSparcityPrior
import tensorflow as tf
import numpy as np

test = np.random.rand(300,100)
c_init = np.full((300,300), 0.05, dtype=np.float32)
# c_init = np.random.uniform(-0.1, 0.1, (20,20)).astype(np.float32)
np.fill_diagonal(c_init, 0)

# keke = getSparcityPrior(test, C_init=None,optimizer='Adam', lambda1=0.001, lambda2=1000, epochs=5000, learning_rate=0.1,print_step=100)

In [2]:
dsc = DeepSubspaceClustering(test, hidden_dims=[100], C=c_init, learning_rate=0.001)

In [4]:
dsc.train(batch_size=100, epochs=100000, print_step=100)

In [ ]:
from sda import StackedDenoisingAutoencoder

sda = StackedDenoisingAutoencoder(dims=[100],epochs=[1000],activations=['tanh'])